In [7]:
#!pip install torchaudio librosa matplotlib numpy pandas
#!pip install wget


In [ ]:
#!git clone https://github.com/jaywalnut310/vits.git


In [1]:
from TTS.api import TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=True)

/home/ahmet/anaconda3/envs/tts_env/lib/python3.10/site-packages/TTS/api.py:70: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.


/home/ahmet/anaconda3/envs/tts_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 > Using model: xtts


/home/ahmet/anaconda3/envs/tts_env/lib/python3.10/site-packages/TTS/tts/layers/xtts/xtts_manager.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.speakers = torch.lo

In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [38]:
#torch.cuda.empty_cache()

In [9]:
tts.tts_to_file(text="Abi gercekten cok salaksin.",
                file_path="output_5.wav",
                speaker_wav="huly.wav",
                language="tr")



 > Text splitted to sentences.
['Abi gercekten cok salaksin.']
 > Processing time: 0.5636017322540283
 > Real-time factor: 0.21100614975891951


'output_5.wav'

In [24]:
import json
import os

def parse_metadata_with_wavs(metadata_path, wavs_folder):
    """
    Reads a metadata.txt file and aligns transcriptions with actual wav files.
    
    Now includes **speaker names** in the final dictionary.

    Returns:
    - A dictionary {wav_filename: {"text": transcription, "speaker": speaker_name}}
    """
    speaker_name = os.path.basename(os.path.dirname(metadata_path))  # ✅ Extract speaker from folder name

    # ✅ Step 1: Extract Transcriptions (Order Only)
    transcriptions = []
    with open(metadata_path, "r", encoding="utf-8") as f:
        current_text = []
        for line in f:
            line = line.strip()

            if "|" in line:
                # ✅ New entry → Save the last one
                if current_text:
                    transcriptions.append(" ".join(current_text).strip())

                # ✅ Extract transcription only (ignore filename)
                parts = line.split("|", 1)
                current_text = [parts[1].strip()] if len(parts) > 1 else []
            else:
                # ✅ Continuation of the previous line
                current_text.append(line.strip())

        # ✅ Save the last transcription
        if current_text:
            transcriptions.append(" ".join(current_text).strip())

    # ✅ Step 2: Extract `.wav` filenames in order
    wav_files = sorted([f for f in os.listdir(wavs_folder) if f.endswith(".wav")])

    # ✅ Step 3: Zip & Align
    if len(wav_files) != len(transcriptions):
        print(f"⚠ Warning: Mismatch in number of WAV files ({len(wav_files)}) vs transcriptions ({len(transcriptions)}) in {metadata_path}")

    aligned_data = {wav: {"text": transcript, "speaker": speaker_name} for wav, transcript in zip(wav_files, transcriptions)}

    return aligned_data




In [25]:
import json
import os

# ✅ Root directory for processed dataset
ROOT_DIR = "m_ailabs_temp/de"
OUTPUT_FILE = "m_ailabs_transcriptions_with_speakers.json"

# ✅ Store all speakers' data
all_speakers_data = {}

# ✅ Loop over each speaker's folder
for speaker_folder in os.listdir(ROOT_DIR):
    speaker_path = os.path.join(ROOT_DIR, speaker_folder)

    # ✅ Ensure it's a folder
    if not os.path.isdir(speaker_path):
        continue

    # ✅ Find all metadata.txt and corresponding wavs folders
    metadata_files = {}
    wav_folders = {}

    for item in os.listdir(speaker_path):
        item_path = os.path.join(speaker_path, item)

        if item.endswith("_metadata.txt"):  # ✅ Identify metadata files
            situation_name = item.replace("_metadata.txt", "")  # Extract situation name
            metadata_files[situation_name] = item_path

        elif item.endswith("_wavs"):  # ✅ Identify wav folders
            situation_name = item.replace("_wavs", "")  # Extract situation name
            wav_folders[situation_name] = item_path

    # ✅ Now we process each situation (aligning metadata with wavs)
    for situation, metadata_file in metadata_files.items():
        if situation in wav_folders:  # ✅ Ensure matching wavs exist
            wavs_folder = wav_folders[situation]
            speaker_data = parse_metadata_with_wavs(metadata_file, wavs_folder)
            all_speakers_data.update(speaker_data)

print(f"✅ Processed {len(all_speakers_data)} audio files across all speakers.")

# ✅ Save full dataset to JSON
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(all_speakers_data, f, ensure_ascii=False, indent=4)

print(f"✅ All speakers' transcriptions saved to: {OUTPUT_FILE}")


✅ Processed 82017 audio files across all speakers.
✅ All speakers' transcriptions saved to: m_ailabs_transcriptions_with_speakers.json
